### Initialise workspace from Azure ML workspace

In [1]:
from azureml.core import Workspace

source_dir = "./source_dir/"
ws = Workspace.from_config(source_dir + "config.json")
print(ws)

Workspace.create(name='Commonlit-Deployment-Space', subscription_id='9d7c60f0-d3e1-49c3-b70b-080e861838c1', resource_group='Commonlit-Deployment')


### Register RoBERTa transformer model

In [2]:
from azureml.core.model import Model

# Register model
model = Model.register(workspace = ws, 
                       model_name = "Commonlit-RoBERTa-Base",
                       model_path = "./outputs/Commonlit-RoBERTa-Base",
                       model_framework = "Custom")

Registering model Commonlit-RoBERTa-Base


### Create environment with necessary dependencies

In [15]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment(name = "Commonlit-Bi-LSTM")
conda_dep = CondaDependencies()

conda_dep.add_conda_package("numpy")

conda_dep.add_pip_package("transformers")
conda_dep.add_pip_package("torch")

conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azureml")
conda_dep.add_pip_package("azureml-contrib-functions")

env.python.conda_dependencies = conda_dep

### Create inference configuration using scoring function

In [16]:
inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./RoBERTa_score.py",
)

### Create Azure Container Instance and deploy model to container

In [18]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(
    ws,
    "commonlit-roberta-base",
    [model],
    inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-31 04:40:02+00:00 Creating Container Registry if not exists.
2021-08-31 04:40:02+00:00 Registering the environment.
2021-08-31 04:40:05+00:00 Use the existing image.
2021-08-31 04:40:05+00:00 Generating deployment configuration.
2021-08-31 04:40:07+00:00 Submitting deployment to compute..
2021-08-31 04:40:11+00:00 Checking the status of deployment commonlit-roberta-base..
2021-08-31 04:45:44+00:00 Checking the status of inference endpoint commonlit-roberta-base.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Endpoint consumption - Test

In [19]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    'data' : ['Dotty continued to go to Mrs. Gray\'s every night with the milk. Sometimes Katie went with her, and then they always paused a while under the acorn-tree and played "King and Queen." Dotty said she wished they could ever remember to bring their nipperkins, for in that case the milk would taste a great deal more like nectar. The "nipperkins" were a pair of handled cups which the children supposed to be silver, and which they always used at table.\nDotty knew she was doing wrong every time she played "King and Queen." She knew the milk was not hers, but Mrs. Gray\'s; still she said to herself, "Ruthie needn\'t give so much measure, all pressed down and run over. If Queenie and I should drink a great deal more, there would always be a quart left. Yes, I know there would."\nMrs. Gray never said anything about the milk; she merely poured it out in a pan, and gave back the pail to Dotty, asking her at the same time as many questions as the child would stay to hear.',
             'Cell division is the process by which a parent cell divides into two or more daughter cells. Cell division usually occurs as part of a larger cell cycle.\n In eukaryotes, there are two distinct types of cell division: a vegetative division, whereby each daughter cell is genetically identical to the parent cell (mitosis), and a reproductive cell division, whereby the number of chromosomes in the daughter cells is reduced by half, to produce haploid gametes (meiosis). \nMeiosis results in four haploid daughter cells by undergoing one round of DNA replication followed by two divisions: homologous chromosomes are separated in the first division, and sister chromatids are separated in the second division.\nBoth of these cell division cycles are used in sexually reproducing organisms at some point in their life cycle, and both are believed to be present in the last eukaryotic common ancestor. Prokaryotes also undergo a vegetative cell division known as binary fission, where their genetic material is segregated equally into two daughter cells. All cell divisions, regardless of organism, are preceded by a single round of DNA replication.']
}

body = str.encode(json.dumps(data))

url = 'http://bf6cc669-3f4a-4dcb-a2f5-4a7c839c6172.australiaeast.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'[-0.3461618423461914, -2.693899631500244]'
